In [14]:
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import dlib
import cv2
import random
import math

In [15]:
Classes=["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]

output="D:\\Research\\multiobject-tracking-dlib\\town_output.avi"

# load our serialized model from disk
print("Loading model...")
net = cv2.dnn.readNetFromCaffe("D:\\Research\\multiobject-tracking-dlib\\mobilenet_ssd\\MobileNetSSD_deploy.prototxt", 
                              "D:\\Research\\multiobject-tracking-dlib\\mobilenet_ssd\\MobileNetSSD_deploy.caffemodel")

Loading model...


In [16]:
def estimateSpeed(location1, location2):
    d_pixels=math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    # ppm=location2[2] / carWidht
    ppm=8.8
    d_meters=d_pixels / ppm
    #print("d_pixels=" + str(d_pixels), "d_meters=" + str(d_meters))
    fps=18
    speed=d_meters * fps * 3.6
    return speed

In [18]:
vs=cv2.VideoCapture("D:\\Research\\multiobject-tracking-dlib\\TownCentreXVID.avi")
#vs=cv2.VideoCapture("D:\\Research\\multiobject-tracking-dlib\\town-center.mp4")
writer=None
speed=[None]*1000

trackers=[]
labels=[]

fps=FPS().start()
while True:    
    (grabbed,frame)=vs.read()
    if frame is None:
        break
        
    frame=imutils.resize(frame,width=600)
    #print(frame.shape)
    #frame = cv2.resize(frame, (600,480))
    rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    if output is not None and writer is None:
        fourcc=cv2.VideoWriter_fourcc(*"MJPG")
        writer=cv2.VideoWriter(output,fourcc,30,(frame.shape[1],frame.shape[0]),True)
        
    if len(trackers)==0:
        (h,w)=frame.shape[:2]
        blob=cv2.dnn.blobFromImage(frame,0.007843,(w,h),127.5)
            
        net.setInput(blob)
        detections=net.forward()
            
        for i in np.arange(0, detections.shape[2]):
            confidence=detections[0,0,i,2]
                
            if confidence>0:
                idx=int(detections[0,0,i,1])
                label=Classes[idx]
                    
                #if class label is not person, ignore
                if Classes[idx]!="person":
                    continue
                    
                box=detections[0,0,i,3:7]*np.array([w,h,w,h])
                (startX,startY,endX,endY)=box.astype("int")
                avs=list((startX,startY))
                evs=list((endX,endY))

                #rectangle
                t=dlib.correlation_tracker()
                rect = dlib.rectangle(int(startX), int(startY), int(endX), int(endY))
                t.start_track(rgb,rect)
                
                speed=estimateSpeed(avs,evs)
                    
                labels.append(label)
                trackers.append(t)
                    
                cv2.rectangle(frame,(startX,startY),(endX,endY),(0,255,0),2)
                cv2.putText(frame,label,(startX,startY-15),cv2.FONT_HERSHEY_SIMPLEX,0.45,
                                (0,255,0),2)
                    
    else:
        for (t,l) in zip(trackers,labels):
            t.update(rgb)
            pos=t.get_position()
                        
            startX=int(pos.left())
            startY=int(pos.top())
            endX=int(pos.right())
            endY=int(pos.bottom())
                        
            cv2.rectangle(frame,(startX,startY),(endX,endY),(0,255,0),2)
            cv2.putText(frame,l,(startX,startY-15),cv2.FONT_HERSHEY_SIMPLEX,0.45,
                        (0,255,0),2)
            cv2.putText(frame, str(speed/462)[:5],(0,20),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,0,0),2,
                        cv2.LINE_AA)
                
    if writer is not None:
        writer.write(frame)

                
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) & 0xFF
                
    if key==ord('q'):
        break
                
    fps.update()
fps.stop()
if writer is not None:
    writer.release()
cv2.destroyAllWindows()
vs.release()
print('Elapsed Time {:.2f}'.format(fps.elapsed()))
print('Approx FPS {:.2f}'.format(fps.fps()))

Elapsed Time 7.74
Approx FPS 12.41
